# Quick Start

## Getting Started

In [1]:
import folium

In [2]:
m = folium.Map(location=[45, -122])

In [3]:
m

In [5]:
folium.Map(location=[45.5236, -122.6750], tiles="Stamen Toner")

## Markers

There are numerous marker types, starting with a simple Leaflet style location marker with a popup and tooltip `HTML`.

In [8]:
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)

tooltip = 'Click me!'

folium.Marker([45.3288, -121.6625], tooltip=tooltip).add_to(m)
folium.Marker([45.3311, -121.7113], tooltip=tooltip).add_to(m)

m

Marker Colors and Icon types

In [16]:
m = folium.Map(
    location=[45.372, -122.6750],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    tooltip="Click",
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)

m

Enable `lat`, `lon` popups

In [17]:
m = folium.Map(
    location=[46.1991, -122.1889],
    tiles='Stamen Terrain',
    zoom_start=13
)

m.add_child(folium.LatLngPopup())

m

In [18]:
m = folium.Map(
    location=[46.8527, -121.7649],
    tiles='Stamen Terrain',
    zoom_start=13
)

folium.Marker(
    [46.8354, -121.7325],
    popup='Camp Muir'
).add_to(m)

m.add_child(folium.ClickForMarker(popup='Waypoint'))

m

## Choropleth maps

Choropleth can be easily created by binding the data between Pandas DataFrames/Series and Geo/TopoJSON geometries. [Color Brewer](http://colorbrewer2.org/#type=sequential&scheme=RdPu&n=3) sequential color schemes are built-in to the library, and can be passed to quickly visualize different combinations.

In [23]:
import branca
import numpy as np
import pandas as pd

In [25]:
# state_geo = os.path.join('data', 'us-states.json')

# state_unemployment = os.path.join('data', 'US_Unemployment_Oct2012.csv')
# state_data = pd.read_csv(state_unemployment)

# m = folium.Map(location=[48, -102], zoom_start=3)

# m.choropleth(
#     geo_data=state_geo,
#     name='choropleth',
#     data=state_data,
#     columns=['State', 'Unemployment'],
#     key_on='feature.id',
#     fill_color='YlGn',
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='Unemployment Rate (%)'
# )


# folium.LayerControl().add_to(m)

# m

In [30]:
import numpy as np
import geohash
import geojson
import folium
from branca import utilities
from folium import GeoJson
import utils

In [27]:
geohash.bbox("w23bpu")

{'s': 1.42822265625,
 'w': 104.051513671875,
 'n': 1.4337158203125,
 'e': 104.0625}

In [32]:
def vis(df, val, key = 'geohash', opacity = 0.75, zoom = 10, scale = 'linear'):
    gs = df.loc[:, key].drop_duplicates()
    lats = []
    lngs = []
    geo_list = []
    for g in gs:  # geohashes `w21je1`
        box = geohash.bbox(g)
        geo_list.append(geojson.Feature(
            properties = {key: g},
            geometry = geojson.Polygon(
                coordinates = [[
                    (box['w'], box['s']),
                    (box['w'], box['n']),
                    (box['e'], box['n']),
                    (box['e'], box['s']),
                    (box['w'], box['s'])
            ]])
        ))
        lngs.append((box['e'] + box['w'])/2)
        lats.append((box['n'] + box['s'])/2)
    geo = geojson.FeatureCollection(geo_list)
    tf_name = '{}.json'.format(1)  # np.random.uniform()
    tf = open(tf_name, 'w')
    geojson.dump(geo, tf)
    tf.close()

    scales = (
        list(df.loc[:, val].quantile([0, 0.2,0.4,0.6,0.8, 1]).drop_duplicates()) if scale == 'quantile'
        else list(np.linspace(0, df.loc[:, val].max(), 6)))
    
    m = folium.Map(location=[sum(lats)/len(lats), sum(lngs)/len(lngs)], zoom_start=zoom, tiles='Stamen Terrain')
    m.choropleth(geo_data=tf_name, data=df,
             columns=[key, val],
             key_on='feature.properties.' + key,
             fill_color='Blues', fill_opacity=opacity, line_opacity=0.1,
             threshold_scale = scales)
    return(m)

In [31]:
geo_code = pd.read_csv("data/geo_id.csv")
sg_geo = geo_code[geo_code["city_id"]==utils.city_id["singapore"]]
sg_geo.head()

,geohash,city_id,country_id,geo_id
546941,w23bpu,6,4,1
546942,w23bpb,6,4,2
546943,w23bn2,6,4,3
546944,w23bjb,6,4,4
546945,w23bj9,6,4,5


In [33]:
vis(sg_geo, val="geo_id")

/Users/jeremy.zhang/anaconda3/lib/python3.6/site-packages/folium/folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
